In [1]:
import csv
import pandas
import urllib
import json
import time
import operator
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import numpy as np
from scipy import sparse
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import re
import string
import pickle

[nltk_data] Downloading package stopwords to /Users/HJK-
[nltk_data]     BD/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class Textloader(object):
    def __init__(self, filename='songcleaned2.csv',embedding_size=100):
        self.name = filename
        self.embedding_size = embedding_size
    
    def __iter__(self):
        with open(self.name, 'r') as f:
            f_csv = csv.reader(f)
            headers = next(f_csv)
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            for row in f_csv:
                lyric = word_list_clean(row[2].replace("\n","").split(" "), regex)
                lyric = [x for x in lyric if x != '']
                yield lyric
                
def word_list_clean(word_list,regex):
    clean_list = []
    for word in word_list:
        word = word.lower()
        word = regex.sub('', word)
        clean_list.append(word)
    return(clean_list)

In [3]:
# loader = Textloader()
# model = gensim.models.Word2Vec(loader, workers=4,size=loader.embedding_size, window=5, min_count=2)
# model.save('word2vector3.model')
# print('ok')

ok


In [4]:
y2 = model.wv.most_similar("love", topn=20)  # 20个最相关的
print(u"和love最相关的词有：\n")
for item in y2:
    print( item[0], item[1])
print( "--------\n")

和love最相关的词有：

oh 0.5569515228271484
true 0.5471696853637695
you 0.5312179327011108
baby 0.5250938534736633
me 0.5224686861038208
loving 0.520522952079773
life 0.5177774429321289
it 0.506965160369873
hope 0.5009456276893616
yeah 0.4938737154006958
that 0.49084243178367615
believe 0.4901423454284668
lovin 0.47710004448890686
lover 0.4753359854221344
christess 0.47528076171875
need 0.469929039478302
know 0.4580895006656647
heart 0.45634105801582336
yes 0.4467881917953491
forever 0.4463220536708832
--------



In [5]:
def to_matrix(word_list):
    matrix = []
    for word in word_list:
        if word in model.wv.vocab:
            matrix.append(np.ndarray.tolist(model.wv.__getitem__(word)))
    return matrix

def word_vector(filename):
    with open(filename, 'r') as f:
            f_csv = csv.reader(f)
            headers = next(f_csv)
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            count = 0
            for row in f_csv:
                lyric = word_list_clean(row[2].replace("\n","").split(" "), regex)
                lyric = [x for x in lyric if x != '']
                matrix = to_matrix(lyric)
                wfilename = "matrix3/"+str(count)+".txt"
                file_writer(matrix, wfilename)
                count += 1
                
def file_writer(data, filename):
     with open(filename, "wb") as fp:   #Pickling
        pickle.dump(data, fp)

model = gensim.models.Word2Vec.load('word2vector3.model')
word_vector("songcleaned2.csv")

In [ ]:
def model(data, train=False):
    """The Model definition."""
    # 2D convolution, with 'SAME' padding (i.e. the output feature map has
    # the same size as the input). Note that {strides} is a 4D array whose
    # shape matches the data layout: [image index, y, x, depth].
    conv = tf.nn.conv2d(data,
                        conv1_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    conv = tf.nn.conv2d(pool,
                        conv2_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, fc2_weights) + fc2_biases

# Training computation: logits + cross-entropy loss.
    logits = model(train_data_node, True)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=train_labels_node, logits=logits))

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                  tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
    # Add the regularization term to the loss.
    loss += 5e-4 * regularizers

    # Optimizer: set up a variable that's incremented once per batch and
    # controls the learning rate decay.
    batch = tf.Variable(0, dtype=data_type())
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch * BATCH_SIZE,  # Current index into the dataset.
      train_size,          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate,
                                         0.9).minimize(loss,
                                                       global_step=batch)

    # Predictions for the current training minibatch.
    train_prediction = tf.nn.softmax(logits)

    # Predictions for the test and validation, which we'll compute less often.
    eval_prediction = tf.nn.softmax(model(eval_data))

In [ ]:
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
        raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
        else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
            predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions